# LangChain
Here are a few patterns where LLMs can be augmented with other systems: 

* Convert natural language to SQL, run the SQL on a database, analyze and present the results
* Call an external webhook or API based on the user query
* Synthesize outputs from multiple models, or chain the models in a specific order

![alt text](images/Patterns_augmenting_LLMs_with_external_systems.jpg "Title")

It may look trivial to plumb these calls together and orchestrate them but it becomes a mundane task to write glue code again and again e.g. for every different data connector or a new model. That’s where LangChain comes in!

LangChain’s modular implementation of components and common patterns combining these components makes it easier to build complex applications based on language models. LangChain enables these models to connect to data sources and systems as agents to take action. 

**Components** are abstractions that work to bring external data, such as your documents, databases, applications,APIs to language models

**Agents** enable language models to communicate with its environment, where the model then decides the next action to take.

LangChain offers a variety of modules that can be used to create language model applications. These modules can be combined to create more complex applications, or can be used individually for simpler applications.
![alt text](images/Figure-3-LangChain_Concepts.max-1300x1300.png "Title")

**Models** are the building block of LangChain providing an interface to different types of AI models. Large Language Models (LLMs), Chat and Text Embeddings models are supported model types.

**Prompts** refers to the input to the model, which is typically constructed from multiple components. LangChain provides interfaces to construct and work with prompts easily - Prompt Templates, Example Selectors and Output Parsers. 

**Memory** provides a construct for storing and retrieving messages during a conversation which can be either short term or long term.

**Indexes** help LLMs interact with documents by providing a way to structure them. LangChain provides Document Loaders to load documents, Text Splitters to split documents into smaller chunks, Vector Stores to store documents as embeddings, and Retrievers to fetch relevant documents.

**Chains** let you combine modular components (or other chains) in a specific order to complete a task.

**Agents** are a powerful construct in LangChain allowing LLMs to communicate with external systems via Tools and observe and decide on the best course of action to complete a given task.
* Tools are functions that perform specific duties, such as Google Search, database lookups, or Python REPL. They take a string as input and return a string as output.

* Agents are responsible for determining which actions to take and in what order. They can use tools, observe the output, or return to the user.

* Toolkits are collections of tools that can be utilized by agents. LangChain supports various toolkits to help developers create powerful applications.


Enterprise data is often stored in SQL databases.

LLMs make it possible to interact with SQL databases using natural langugae.

LangChain offers SQL Chains and Agents to build and run SQL queries based on natural language prompts.

These are compatible with any SQL dialect supported by SQLAlchemy (e.g., MySQL, PostgreSQL, Oracle SQL, Databricks, SQLite).

They enable use cases such as:
* Generating queries that will be run based on natural language questions
* Creating chatbots that can answer questions based on database data
* Building custom dashboards based on insights a user wants to analyze

## Overview
LangChain provides tools to interact with SQL Databases:

1. Build SQL queries based on natural language user questions
2. Query a SQL database using chains for query creation and execution
3. Interact with a SQL database using agents for robust and flexible querying

![alt text](images/sql_usecase-d432701261f05ab69b38576093718cf3.png "Title")

In [ ]:
# !pip install langchain==0.0.265
# !pip install openai==0.27.8

In [1]:
# import langchain
# import openai
from langchain.llms import OpenAI, VertexAI
from langchain.agents import AgentExecutor, create_sql_agent 
from langchain.agents.agent_toolkits import SQLDatabaseToolkit 
from langchain.agents.agent_types import AgentType
# from langchain.sql_database import SQLDatabase
from google.cloud.sql.connector import Connector
from config import config

In [2]:
from importlib.metadata import version
print("langchain version:", version('langchain'))
print("openai version:", version('openai'))

langchain version: 0.0.265
openai version: 0.27.8


In [3]:
#setup & init env vars
PROJECT_ID="jared-playground"
REGION="us-central1"
INSTANCE_NAME="gen-ai-text-to-mysql"
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance

#Just double check stuff....
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

!gcloud config set project $PROJECT_ID
!gcloud config get project
!gcloud auth list

Your instance connection name is: jared-playground:us-central1:gen-ai-text-to-mysql
Updated property [core/project].
jared-playground
                  Credentialed Accounts
ACTIVE  ACCOUNT
*       555626409143-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [4]:
def get_db():
    # initialize Connector object
    connector = Connector()
    
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pymysql",
            user=DB_USER,
            db=DB_NAME
        )
        return conn
    
    db = SQLDatabase.from_uri(
        "mysql+pymysql://",
        engine_args = {'creator': getconn},
    )
    return db

LangChain has an SQL Agent which provides a more flexible way of interacting with SQL Databases than the ```SQLDatabaseChain```.

The main advantages of using the SQL Agent are:
* It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table)
*It can recover from errors by running a generated query, catching the traceback and regenerating it correctly
To initialize the agent, we use ```create_sql_agent``` function.

This agent contains the ```SQLDatabaseToolkit``` which contains tools to:

Create and execute queries
Check query syntax
Retrieve table descriptions
... and more


Zero Shot ReAct
The agent used here is a "zero-shot-react-description" agent. Zero-shot means the agent functions on the current action only — it has no memory. It uses the ReAct framework to decide which tool to use, based solely on the tool’s description.


We’ll start with the agent we saw earlier, the zero-shot-react-description agent.

As described earlier, we use this agent to perform “zero-shot” tasks on some input. That means the agent considers one single interaction with the agent — it will have no memory.

The zero-shot agent works well but lacks conversational memory. This lack of memory can be problematic for chatbot-type use cases that need to remember previous interactions in a conversation.

In [5]:
def test_sql_agent(db, llm, question):
    
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        return_intermediate_steps=True,
        use_query_checker=True,
        max_iterations=5
    )
    # agent_executor.return_intermediate_steps(question)
    agent_executor.run(question)
    
    
    # use_query_checker=True

In [6]:
PROMPT = """ 
The first three digits of the phone number is the area code. The phone number is in the customers table. Revenue is in the payments table. What are the top 10 area codes that generated the most revenue filtered by customers in the USA?
"""

test_sql_agent(
    db = get_db(),
    llm = VertexAI(model_name="code-bison@001", temperature=0, verbose=True),
    # question = "How many orders were placed in the year 2005?"
    question = PROMPT
)



> Entering new AgentExecutor chain...
Question: The first three digits of the phone number is the area code. The phone number is in the customers table. Revenue is in the payments table. What are the top 10 area codes that generated the most revenue filtered by customers in the USA?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought:The tables "customers" and "payments" seem relevant. I will query the schema for these tables.
Action: sql_db_schema
Action Input: customers, payments
Observation: 
CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addr

We can see:
* The agent is using a ReAct style prompt
* First, it will look at the tables: Action: ```sql_db_list_tables``` using tool ```sql_db_list_tables```
* Given the tables as an observation, it ```thinks``` and then determinates the next ```action```:
    * Observation: ...
    * Thought: ...
    * Action: ...
    * Action Input: ...
* It then formulates the query using the schema from tool ```sql_db_schema```
* It finally executes the generated query using tool ```sql_db_query```
* If the query is successful it returns:
    * Thought: I now know the final answer.
    * Final Answer: ...

In [7]:
test_sql_agent(
    db = get_db(),
    llm = OpenAI(model_name="text-davinci-003", temperature=0, verbose=True, openai_api_key=config.openai_api_key),
    question = "How many orders were placed in the year 2005?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought: I should query the schema of the orders table.
Action: sql_db_schema
Action Input: orders
Observation: 
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10102	2003-01-10	2003-01-18	2003-01-14	Shipped	None	181
*/
Thought: I should qu

In [8]:
db = get_db()
print(db.table_info)


CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)DEFAULT CHARSET=latin1 ENGINE=InnoDB

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmitt	Carine 	40.32.2555	54, rue Royale	None	Nantes	None	44000	France	1370	21000.00
112	Signal Gift Stores	King	Jean	7025551838	8489 Strong St

The query builder can be improved in several ways, such as (but not limited to):

* Customizing database description to your specific use case.
* Hardcoding a few examples of questions and their corresponding SQL query in the prompt.
* Using a vector database to include dynamic examples that are relevant to the specific user question.

All these examples involve customizing the chain's prompt. For example, we can include a few examples in our prompt like so:

In [9]:
from langchain.prompts.prompt import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Be sure to put a semi-colon after query.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that corrsespond to questions are:

{few_shot_examples}

Question: {input}"""
CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"], template=TEMPLATE
)

In [10]:
few_shot_examples = """
Here is an example: 
The question is: How many unique customers placed an order between 1/1/2004 and 1/31/2004? 
The corresponding SQL is: SELECT count(distinct t1.customerNumber) as distinct_customers FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber WHERE t5.orderDate BETWEEN '2004-01-01' AND '2004-01-31';
Here is an example: 
The question is: What is the product name and price of the highest priced item that was ever purchased? 
The corresponding SQL is: SELECT t8.productName, t4.priceEach FROM orders t5 INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber LEFT JOIN products t8 ON t4.productCode = t8.productCode ORDER BY priceEach DESC LIMIT 1;
Here is an example: 
The question is: What is the name of the customer with the highest order amount on 12/1/2003? 
The corresponding SQL is: SELECT orders.customerName, orders.total_orderAmount FROM (SELECT t5.orderNumber, t1.customerNumber, t1.customerName, SUM(t4.quantityOrdered*t4.priceEach) as total_orderAmount FROM customers t1 INNER JOIN orders t5 ON t1.customerNumber = t5.customerNumber INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber WHERE t5.orderDate = '2003-12-01' GROUP BY 1,2,3) orders ORDER BY total_orderAmount DESC LIMIT 1;
Here is an example: 
The question is: How many unique orders were placed at the Paris office during the year 2004?
The corresponding SQL is: SELECT count(distinct t5.orderNumber) as orders FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber LEFT JOIN employees t2 ON t1.salesRepEmployeeNumber = t2.employeeNumber LEFT JOIN offices t3 ON t2.officeCode = t3.officeCode WHERE YEAR(t5.orderDate) = 2004 AND t3.city = 'Paris';
"""

question = """ 
The phone number is in the customers table. 
Extract the first three digits of the phone number for the area code.
Revenue is in the payments table. 
What are the top 10 area codes that generated the most revenue filtered by customers in the USA?
"""

table_info = """
payments, customers
"""

In [11]:
db = get_db()
llm = VertexAI(model_name="code-bison@001", temperature=0, verbose=True)
toolkit = SQLDatabaseToolkit(db=db, llm=llm, prompt=CUSTOM_PROMPT)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps=True,
    use_query_checker=True,
    max_iterations=5
)
agent_executor.run(
    dict(input=question, 
         few_shot_examples=few_shot_examples,
         table_info=db.get_table_info(), 
         dialect=db.dialect
        )
)




> Entering new AgentExecutor chain...
Question: The phone number is in the customers table. 
Extract the first three digits of the phone number for the area code.
Revenue is in the payments table. 
What are the top 10 area codes that generated the most revenue filtered by customers in the USA?

Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought:The customers table has the phone number and the country.  The payments table has the revenue.  I need to join the tables on the customer id.
Action: sql_db_schema
Action Input: customers, payments
Observation: 
CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(5

"[('4155551450', Decimal('584188.24')), ('2125557413', Decimal('177913.95')), ('6175558555', Decimal('164079.02')), ('6505551386', Decimal('132340.78')), ('6035"